<a href="https://colab.research.google.com/github/OutputGEO/censo2022/blob/main/Territorios_Indigenas_Brasil_WFS_IBGE_semTOKEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import folium
from IPython.display import display
!pip install PyGithub
from github import Github
import os

# Função para adicionar um banner no topo
def add_banner(map_obj, title, legend):
    banner_html = """
    <div style="
        position: absolute;
        top: 0;
        left: 0;
        width: 100%;
        text-align: left;
        padding: 10px;
        z-index: 1000;
        font-family: 'Arial', sans-serif;
        background-color: #ffffff;
        ">
        <img src="https://raw.githubusercontent.com/OutputGEO/webmap_python/main/banner.png" alt="Banner Image" style="max-width: 100%; max-height: 100px;"> <!-- Define a altura máxima do logo -->
        <h3 style="margin: 5px 0 0 0; font-weight: bold;"><a href="https://www.ibge.gov.br/estatisticas/sociais/populacao/27177-censo-demografico-2022.html" target="_blank">{}</a></h3>
        <p style="margin: 0; font-size: 14px;">{}</p>
    </div>
    """.format(title, legend)
    map_obj.get_root().html.add_child(folium.Element(banner_html))


# Função para adicionar um rodapé com informações da empresa
def add_footer(map_obj, company_info):
    footer_html = """
    <div style="
        position: absolute;
        bottom: 0;
        left: 50%; /* Alinha o rodapé ao centro */
        transform: translateX(-50%); /* Centraliza horizontalmente */
        width: 100%;
        text-align: center; /* Alinha o conteúdo ao centro */
        background-color: #ffffff;
        padding: 10px;
        border-top: 2px solid #000000;
        z-index: 1000;
        ">
        <a href="https://www.outputgeo.com.br" target="_blank"><img src="https://avatars.githubusercontent.com/u/150393907?v=4" alt="OutputGEO Logo" style="max-width: 50px;"></a><br> <!-- Define a largura máxima do logo -->
        <p style="margin: 0;">{}</p>
        <p style="margin: 0;"><a href="mailto:contato@outputgeo.com">contato@outputgeo.com</a> | <a href="https://www.outputgeo.com.br">www.outputgeo.com.br</a></p>
    </div>
    """.format(company_info)
    map_obj.get_root().html.add_child(folium.Element(footer_html))


# URL do arquivo JSON
url = 'https://geoservicoscenso2022.ibge.gov.br/geoserver/censo2022/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=censo2022%3Aqg_2022_610_terraindigena&maxFeatures=50000&outputFormat=application%2Fjson'

# Realiza a solicitação GET
response = requests.get(url)

# Verifica se a solicitação foi bem-sucedida (código de status 200)
if response.status_code == 200:
    # Converte os dados para o formato JSON
    json_data = response.json()

    # Cria um mapa usando folium
    m = folium.Map(location=[-12.836238983426362, -50.62536682519216], zoom_start=5)

    # Adiciona os polígonos ao mapa com a legenda "Setores censitários"
    folium.GeoJson(json_data,
                   name='Territórios Indígenas',
                   popup=folium.GeoJsonPopup(fields=['cd_ti', 	'nm_ti',	'cd_funai',	'nm_funai',	'fase',	'quadro',	'cd_recorte'],
                                              aliases=['Código do Território Indígena: ', 'Denominação do Território Indígena: ', 'Código FUNAI: ', 'Denominação FUNAI: ', 'Fase: ', 'Quadro: ', 'Recorte: '],
                                              localize=True)).add_to(m)





    # Adiciona a camada do mapa de ruas do OSM
    folium.TileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
                     attr='OpenStreetMap',
                     name='Mapa-base - Ruas (OpenStreetMap)',
                     overlay=True).add_to(m)

    # Adiciona a camada de imagem aérea
    folium.TileLayer('https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                     attr='Google Satellite',
                     name='Mapa-base - Imagem Aérea (Google Satellite)',
                     overlay=True).add_to(m)

    # Adiciona banner no topo
    add_banner(m, 'Territórios Indígenas', 'Territórios divulgados nos resultados do Censo Demográfico de 2022 realizado pelo IBGE')

    # Adiciona informações da empresa no rodapé
    add_footer(m, "OutputGEO Consultoria em Geoprocessamento")

    # Adiciona o controle de camadas ao mapa
    folium.LayerControl(position='bottomleft').add_to(m)

    # Exibe o mapa
    display(m)



# Salva o mapa como um arquivo HTML temporário no Colab
html_file_path = "territorios_indigenas_br.html"
m.save(html_file_path)

#Inserindo o TOKEN do GTIHUB como variavel do ambiente
os.environ['GITHUB_TOKEN'] = 'INSIRA AQUI O TOKEN PESSOAL DO GITHUB'

token = os.environ.get('GITHUB_TOKEN')


# Autenticação no GitHub
g = Github("geooutput@gmail.com", token)  # Substitua pelo seu email e token de acesso pessoal

# Seleciona o repositório
repo = g.get_user().get_repo("censo2022")  # Substitua pelo nome do seu repositório no GitHub

# Nome do arquivo no GitHub
file_name = "territorios_indigenas_br.html"

# Lê o conteúdo do arquivo HTML local
try:
    with open(html_file_path, "r") as file:
        content = file.read()
except FileNotFoundError:
    print("Arquivo HTML não encontrado. Verifique o caminho fornecido.")

# Obtém o SHA do arquivo existente, se existir
try:
    existing_file = repo.get_contents(file_name)
    sha = existing_file.sha
except Exception as e:
    sha = None

# Descrição do commit
commit_message = "Atualizando arquivo HTML com o mapa gerado"

# Tenta criar ou atualizar o arquivo no repositório
try:
    if sha:
        repo.update_file(file_name, commit_message, content, sha, branch="main")
        print("Arquivo atualizado com sucesso no repositório do GitHub!")
    else:
        repo.create_file(file_name, commit_message, content, branch="main")
        print("Arquivo criado com sucesso no repositório do GitHub!")
except Exception as e:
    print("Ocorreu um erro ao criar ou atualizar o arquivo no GitHub:", e)



